In [2]:
import numpy as np
import pandas as pd
import warnings
import shutil
import os


import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras.layers import LeakyReLU, Dropout, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, Model
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, TensorBoard



warnings.filterwarnings("ignore")

### Utils



In [3]:
def display_img(img_id):
    PATH_TO_IMGS = "CelebA_FR_Triplets/images/"

    img = mpimg.imread(PATH_TO_IMGS + img_id)
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    


def preprocess_img(img):
    # Reshape the image to (100, 100, 3)
    img = tf.image.resize(img, [100, 100])
    # Rescale the image values to be between 0 and 1
    
    std = np.std(img)
    mean = np.mean(img)
    
    
    
    return (img - mean) / std

### Images flow & Data Augmentation

In [4]:
train_datagen = ImageDataGenerator(
                        preprocessing_function=preprocess_img,
                        rotation_range = 15,
                        brightness_range = [0.6, 1],
                        zoom_range = 0.1,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        horizontal_flip=True,
                        
                    )

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_img)




train_generator = train_datagen.flow_from_directory("train/",
                                                    class_mode="binary",
                                                    target_size=(100, 100),
                                                    batch_size=32)

val_generator = val_datagen.flow_from_directory("validation/",
                                                class_mode="binary",
                                               target_size=(100, 100),
                                                batch_size=32)

Found 40000 images belonging to 2 classes.
Found 3250 images belonging to 2 classes.


## Design Architecture of NN

In [5]:

class MLflowLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        for metric, value in logs.items():
            mlflow.log_metric(metric, value, step=epoch)



checkpoint_filepath = 'checkpoints/baselinenn_{epoch:02d}_{val_accuracy:04f}.h5'


model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,  # Set to False to save the entire model
    save_best_only=True,     # Set to False to save the model after every epoch regardless of performance
    verbose=1                 # Logs a message whenever a model is saved
)


tensorboard = TensorBoard(log_dir="logs/", update_freq="batch")

In [6]:
nasnet_base = tf.keras.applications.NASNetMobile(weights="imagenet", include_top=False,input_shape=(100, 100, 3))

#nasnet_base.trainable = False

for l in nasnet_base.layers[:-15]:
    l.trainable = False


# Add new layers on top of the model
x = nasnet_base.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x) 
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1, activation="sigmoid")(x)  # Add a Dense layer with a sigmoid activation for binary classification

# Create a new model
model = Model(inputs=nasnet_base.input, outputs=x)

# Compile the model

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

model.compile(optimizer=Adam(0.01), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary to verify the new architecture
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 stem_conv1 (Conv2D)         (None, 49, 49, 32)           864       ['input_1[0][0]']             
                                                                                                  
 stem_bn1 (BatchNormalizati  (None, 49, 49, 32)           128       ['stem_conv1[0][0]']          
 on)                                                                                              
                                                                                                  
 activation (Activation)     (None, 49, 49, 32)           0         ['stem_bn1[0][0]']        

 _right2_stem_1 (SeparableC                                         on_right2_stem_1[0][0]']      
 onv2D)                                                                                           
                                                                                                  
 separable_conv_1_pad_reduc  (None, 53, 53, 32)           0         ['activation_7[0][0]']        
 tion_right3_stem_1 (ZeroPa                                                                       
 dding2D)                                                                                         
                                                                                                  
 activation_9 (Activation)   (None, 25, 25, 11)           0         ['reduction_add_1_stem_1[0][0]
                                                                    ']                            
                                                                                                  
 separable

 reduction_right5_stem_1 (M  (None, 25, 25, 11)           0         ['reduction_pad_1_stem_1[0][0]
 axPooling2D)                                                       ']                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 50, 50, 32)           0         ['adjust_relu_1_stem_2[0][0]']
 g2D)                                                                                             
                                                                                                  
 reduction_add3_stem_1 (Add  (None, 25, 25, 11)           0         ['reduction_left3_stem_1[0][0]
 )                                                                  ',                            
                                                                     'separable_conv_2_bn_reductio
                                                                    n_right3_stem_1[0][0]']       
          

 ion_right1_stem_2 (BatchNo                                         ight1_stem_2[0][0]']          
 rmalization)                                                                                     
                                                                                                  
 activation_13 (Activation)  (None, 13, 13, 22)           0         ['separable_conv_1_bn_reductio
                                                                    n_left1_stem_2[0][0]']        
                                                                                                  
 activation_15 (Activation)  (None, 13, 13, 22)           0         ['separable_conv_1_bn_reductio
                                                                    n_right1_stem_2[0][0]']       
                                                                                                  
 separable_conv_2_reduction  (None, 13, 13, 22)           1034      ['activation_13[0][0]']       
 _left1_st

                                                                    n_left4_stem_2[0][0]']        
                                                                                                  
 reduction_left2_stem_2 (Ma  (None, 13, 13, 22)           0         ['reduction_pad_1_stem_2[0][0]
 xPooling2D)                                                        ']                            
                                                                                                  
 separable_conv_2_bn_reduct  (None, 13, 13, 22)           88        ['separable_conv_2_reduction_r
 ion_right2_stem_2 (BatchNo                                         ight2_stem_2[0][0]']          
 rmalization)                                                                                     
                                                                                                  
 separable_conv_2_reduction  (None, 13, 13, 22)           1034      ['activation_19[0][0]']       
 _right3_s

...

                                                                                                  
 separable_conv_2_bn_normal  (None, 7, 7, 88)             352       ['separable_conv_2_normal_left
 _left1_8 (BatchNormalizati                                         1_8[0][0]']                   
 on)                                                                                              
                                                                                                  
 separable_conv_2_bn_normal  (None, 7, 7, 88)             352       ['separable_conv_2_normal_righ
 _right1_8 (BatchNormalizat                                         t1_8[0][0]']                  
 ion)                                                                                             
                                                                                                  
 separable_conv_2_bn_normal  (None, 7, 7, 88)             352       ['separable_conv_2_normal_left
 _left2_8 

 adding2D)                                                                                        
                                                                                                  
 separable_conv_1_pad_reduc  (None, 13, 13, 176)          0         ['activation_132[0][0]']      
 tion_right1_reduce_8 (Zero                                                                       
 Padding2D)                                                                                       
                                                                                                  
 separable_conv_1_reduction  (None, 4, 4, 176)            35376     ['separable_conv_1_pad_reducti
 _left1_reduce_8 (Separable                                         on_left1_reduce_8[0][0]']     
 Conv2D)                                                                                          
                                                                                                  
 separable

 )                                                                  n_right2_reduce_8[0][0]']     
                                                                                                  
 separable_conv_1_bn_reduct  (None, 4, 4, 176)            704       ['separable_conv_1_reduction_r
 ion_right3_reduce_8 (Batch                                         ight3_reduce_8[0][0]']        
 Normalization)                                                                                   
                                                                                                  
 separable_conv_1_bn_reduct  (None, 4, 4, 176)            704       ['separable_conv_1_reduction_l
 ion_left4_reduce_8 (BatchN                                         eft4_reduce_8[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 reduction

 reduction_concat_reduce_8   (None, 4, 4, 704)            0         ['reduction_add_2_reduce_8[0][
 (Concatenate)                                                      0]',                          
                                                                     'reduction_add3_reduce_8[0][0
                                                                    ]',                           
                                                                     'add_3[0][0]',               
                                                                     'reduction_add4_reduce_8[0][0
                                                                    ]']                           
                                                                                                  
 adjust_avg_pool_1_9 (Avera  (None, 4, 4, 528)            0         ['adjust_relu_1_9[0][0]']     
 gePooling2D)                                                                                     
          

 )                                                                  eft1_9[0][0]']                
                                                                                                  
 activation_144 (Activation  (None, 4, 4, 176)            0         ['separable_conv_1_bn_normal_r
 )                                                                  ight1_9[0][0]']               
                                                                                                  
 activation_146 (Activation  (None, 4, 4, 176)            0         ['separable_conv_1_bn_normal_l
 )                                                                  eft2_9[0][0]']                
                                                                                                  
 activation_148 (Activation  (None, 4, 4, 176)            0         ['separable_conv_1_bn_normal_r
 )                                                                  ight2_9[0][0]']               
          

In [8]:

model.fit(
            train_generator,
            validation_data=val_generator,
            epochs=15,
            batch_size=32,
           callbacks=[model_checkpoint_callback, tensorboard]
           )




Epoch 1/15
32/32 [==============================] - ETA: 0s - loss: 0.5280 - accuracy: 0.7620
Epoch 1: val_loss improved from inf to 0.43685, saving model to checkpoints/baselinenn_01.h5
32/32 [==============================] - 164s 2s/step - loss: 0.5280 - accuracy: 0.7620 - val_loss: 0.4368 - val_accuracy: 0.7880
Epoch 2/15
32/32 [==============================] - ETA: 0s - loss: 0.3787 - accuracy: 0.8310
Epoch 2: val_loss did not improve from 0.43685
32/32 [==============================] - 69s 2s/step - loss: 0.3787 - accuracy: 0.8310 - val_loss: 0.4971 - val_accuracy: 0.8000
Epoch 3/15
32/32 [==============================] - ETA: 0s - loss: 0.2598 - accuracy: 0.8960
Epoch 3: val_loss improved from 0.43685 to 0.26875, saving model to checkpoints/baselinenn_03.h5
32/32 [==============================] - 72s 2s/step - loss: 0.2598 - accuracy: 0.8960 - val_loss: 0.2687 - val_accuracy: 0.9160
Epoch 4/15
32/32 [==============================] - ETA: 0s - loss: 0.2159 - accuracy: 0.9280

KeyboardInterrupt: 